In [1]:
pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 16.6 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, BitsAndBytesConfig
import transformers
import torch
import json
import gc
import os

# Set memory optimization environment variables
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Clear any existing cache before loading model
torch.cuda.empty_cache()
gc.collect()

model = "Qwen/Qwen2.5-Coder-7B-Instruct"

# Configure 4-bit quantization to reduce memory usage
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model)
# Qwen models typically have proper pad tokens, but add if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    model_kwargs={"quantization_config": quantization_config},
    max_length=4096,  # Set reasonable max length
)

print("Qwen2.5-Coder-7B-Instruct model loaded successfully with memory optimizations!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Device set to use cuda:0


Qwen2.5-Coder-7B-Instruct model loaded successfully with memory optimizations!


In [6]:
system_prompt = """
You are "Pythonix", a research-based Python programming tutor. Your entire methodology is grounded in evidence-based educational frameworks.

## Core Directives
1.  **Ground Your Analysis:** You MUST base all analysis and feedback on the **Code Knowledge Tracing (CodeLKT)** and **Computational Thinking (CT)** frameworks.
2.  **Assess, Don't Assume:** Collect evidence from the code to assess the student's knowledge state across CT dimensions.
3.  **Guide, Don't Solve:** Your purpose is to guide the student to discover the solution through Socratic inquiry. **Never provide direct code solutions or fixes.**

---

## Input Format
You will be given the problem and the student's code using the following format:
**[Problem Description]**
...
**[Student Code]**
...

---

## Analysis & Response Protocol

### Step 1: Internal Analysis (Mandatory)
Before generating a response, you MUST perform a silent, internal analysis. Structure your thoughts exactly as follows:
<thinking>
1.  **Problem Decomposition**: What are the core CT concepts and practices required by the problem (e.g., loops, conditionals, abstraction)?
2.  **Evidence Collection**:
    - **CT Concepts Shown**: [List concepts the student correctly applied, e.g., "Correctly used a `for` loop for iteration"].
    - **CT Practices Shown**: [List practices the student demonstrated, e.g., "Decomposed the problem into a helper function"].
    - **Knowledge Gaps**: [Identify specific CT concepts/practices that are missing or incorrect, e.g., "Missed edge case for empty lists", "Algorithm is O(n^2) where O(n) is possible"].
3.  **Knowledge State Assessment**:
    - **Overall Competence**: [Choose: Novice / Developing / Proficient / Advanced].
    - **Justification**: [Briefly explain your choice based on the evidence].
4.  **Strategic Goal**: Based on the knowledge state, my goal is to [e.g., "clarify the purpose of dictionaries for a Developing learner" or "challenge a Proficient learner to consider algorithmic efficiency"].
</thinking>

### Step 2: Formatted Response to Student
After completing your internal analysis, generate your response to the student using this exact markdown template:

**[Appropriate Greeting]**

### 💡 Evidence Summary
I've analyzed your solution based on the principles of computational thinking. Here's what I see:
* **Demonstrated Competencies**: [Acknowledge 1-2 specific demonstrated skills using CT language. e.g., "You've done an excellent job of using a `for` loop to implement the **sequence** and **iteration** concepts."]
* **Problem-Solving Approach**: [Comment positively on their CT practice. e.g., "Your approach of creating a function shows good **abstraction**."]

### ❓ Diagnostic Inquiry
To help me understand your thought process better, could you tell me:
* [Generate 1-2 adaptive questions based on their assessed knowledge state and your strategic goal. e.g., (For Developing) "Can you walk me through, in plain English, what your code is doing inside the loop?", (For Proficient) "What were the pros and cons of the data structure you chose here?"]

### 🎯 Guided Discovery
Here is a challenge to help you push your solution to the next level:
* [Provide one targeted question aimed at their zone of proximal development. e.g., "Your solution works perfectly for the examples given. What might happen if the input list contained duplicate numbers? How could your algorithm handle that?"]

### 🚀 Next Learning Edge
Based on your work here, a great next step would be to explore **[Recommend the next logical CT concept or practice, e.g., "algorithmic efficiency (Big O notation)" or "advanced dictionary methods"]**.

**[Encouraging sign-off]**

---

## Critical Constraints
- **DO NOT** write, correct, or complete the student's code.
- **DO NOT** give direct answers. Frame everything as a question or a concept to explore.
- **DO NOT** be vague. All feedback must be tied to specific evidence in the code.
"""

In [7]:
def PyGuide(user_prompt, max_new_tokens=1024):  # Reduced from 2048
    """
    Takes a user prompt, wraps it with the system persona using Qwen's chat template,
    and returns the model's cleaned response with memory optimization.
    """
    # Format the conversation using Qwen's chat template
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # Use the tokenizer's chat template to format the messages
    full_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Clear cache before generation
    torch.cuda.empty_cache()

    try:
        sequences = pipeline(
            full_prompt,
            do_sample=True,
            top_k=10,
            temperature=0.1,
            top_p=0.95,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            max_new_tokens=max_new_tokens,
            return_full_text=False,  # Only return generated text
            clean_up_tokenization_spaces=True
        )

        model_response = sequences[0]['generated_text'].strip()

    except Exception as e:
        print(f"Generation error: {e}")
        model_response = f"Error generating response: {str(e)}"

    finally:
        # Always clean up after generation
        torch.cuda.empty_cache()
        gc.collect()

    return model_response

In [8]:
# Memory-optimized evaluation loop
input_file_path = '/content/drive/MyDrive/FOSSEE_Project/Qwen_2.5_Coder/student_codes.json'
output_file_path = '/content/drive/MyDrive/FOSSEE_Project/Qwen_2.5_Coder/evaluation_results.json'
output_results = []

print("Starting automated evaluation with Qwen2.5-Coder-7B-Instruct and memory optimizations...")

# Load the entire JSON file into a dictionary
with open(input_file_path, 'r') as f:
    student_data = json.load(f)

total_items = len(student_data)
print(f"Total items to process: {total_items}")

# Process items one by one with memory cleanup
for idx, (problem_id, content) in enumerate(student_data.items(), 1):
    print(f"\n--- Processing Problem ID: {problem_id} ({idx}/{total_items}) ---")

    # Clear memory before each processing
    torch.cuda.empty_cache()
    gc.collect()

    student_code = content['student_code']
    problem_description = content['problem_description']

    # Construct a more concise user prompt
    current_user_prompt = f"""**[Problem Description]**
{problem_description}

**[Student Code]**
```python
{student_code}
```"""

    try:
        # Get the model's response with error handling
        print("Generating response...")
        model_response = PyGuide(current_user_prompt, max_new_tokens=1024)

        # Print abbreviated results to save console memory
        print(f"✅ Successfully processed {problem_id}")
        print(f"Response length: {len(model_response)} characters")
        print(f"First 200 chars of response: {model_response[:200]}...")

        # Store results
        output_results.append({
            "problem_id": problem_id,
            "problem_description": problem_description,
            "student_code": student_code,
            "model_response": model_response,
            "processing_status": "success"
        })

        # Save intermediate results every 2 items to prevent data loss
        if idx % 2 == 0 or idx == total_items:
            temp_output_path = output_file_path.replace('.json', '_temp.json')
            with open(temp_output_path, 'w') as temp_file:
                json.dump(output_results, temp_file, indent=2)
            print(f"Intermediate save completed ({idx}/{total_items})")

    except Exception as e:
        print(f"❌ Error processing {problem_id}: {str(e)}")
        # Log the error but continue with next item
        output_results.append({
            "problem_id": problem_id,
            "problem_description": problem_description,
            "student_code": student_code,
            "model_response": f"ERROR: {str(e)}",
            "processing_status": "error",
            "error_details": str(e)
        })

    # Force memory cleanup after each item
    torch.cuda.empty_cache()
    gc.collect()

    print(f"Memory cleanup completed for {problem_id}")

# Save the final results
print(f"\nSaving all results to {output_file_path}...")
with open(output_file_path, 'w') as outfile:
    json.dump(output_results, outfile, indent=4)

# Clean up temporary file
temp_output_path = output_file_path.replace('.json', '_temp.json')
if os.path.exists(temp_output_path):
    os.remove(temp_output_path)
    print("Temporary file cleaned up.")

# Final statistics
successful_items = len([r for r in output_results if r.get('processing_status') == 'success'])
error_items = len([r for r in output_results if r.get('processing_status') == 'error'])

print(f"\n✅ Automation complete. All results have been saved.")
print(f"Successfully processed: {successful_items} items")
print(f"Errors encountered: {error_items} items")
print(f"Total items: {len(output_results)}")

# Final memory cleanup
torch.cuda.empty_cache()
gc.collect()
print("Final memory cleanup completed.")

Starting automated evaluation with Qwen2.5-Coder-7B-Instruct and memory optimizations...
Total items to process: 10

--- Processing Problem ID: C01_P01_CountVowels (1/10) ---
Generating response...
✅ Successfully processed C01_P01_CountVowels
Response length: 931 characters
First 200 chars of response: **[Appropriate Greeting]**

### 💡 Evidence Summary
I've analyzed your solution based on the principles of computational thinking. Here's what I see:
* **Demonstrated Competencies**: You've done an exc...
Memory cleanup completed for C01_P01_CountVowels

--- Processing Problem ID: C01_P02_GCD (2/10) ---
Generating response...
✅ Successfully processed C01_P02_GCD
Response length: 937 characters
First 200 chars of response: **[Appropriate Greeting]**

### 💡 Evidence Summary
I've analyzed your solution based on the principles of computational thinking. Here's what I see:
* **Demonstrated Competencies**: You've done an exc...
Intermediate save completed (2/10)
Memory cleanup completed for C01

In [ ]:
# Optional: Check GPU memory usage
if torch.cuda.is_available():
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU Memory Cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    print(f"GPU Memory Summary:")
    print(torch.cuda.memory_summary())

In [ ]:
# Optional: Display a sample of the results
if output_results:
    print("Sample result:")
    sample = output_results[0]
    print(f"Problem ID: {sample['problem_id']}")
    print(f"Status: {sample['processing_status']}")
    print(f"Response preview: {sample['model_response'][:300]}...")

In [ ]:
# Optional: Test the PyGuide function with a simple example
test_prompt = """**[Problem Description]**
Write a function that takes a list of numbers and returns the sum of all even numbers.

**[Student Code]**
```python
def sum_evens(numbers):
    total = 0
    for num in numbers:
        if num % 2 == 0:
            total += num
    return total
```"""

print("Testing PyGuide function...")
test_response = PyGuide(test_prompt)
print("Test Response:")
print(test_response)